In [30]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
import pandas as pd
from nltk.corpus import wordnet as wn 
import csv
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
wordNetLemmatizer = WordNetLemmatizer()
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import joblib

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\keyur\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\keyur\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\keyur\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data = pd.read_csv("../Data/Filtered Data/training.csv")

In [3]:
data.shape

(4920, 174)

In [4]:
data.head()

,Unnamed: 0,vomiting,indigestion,internal_itching,silver_like_dusting,extra_marital_contacts,itching,obesity,joint_pain,brittle_nails,...,Hyperthyroidism,Hypoglycemia,Osteoarthristis,Arthritis,(vertigo) Paroymsal Positional Vertigo,Acne,Urinary tract infection,Psoriasis,Impetigo,disease
0,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Fungal infection
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Fungal infection
2,2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Fungal infection
3,3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Fungal infection
4,4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Fungal infection


In [5]:
data.drop('Unnamed: 0',axis=1, inplace=True)

In [6]:
symp_new = data.columns

In [7]:
X= data[symp_new]
y = data[["disease"]]
np.ravel(y)
print(X)

      vomiting  indigestion  internal_itching  silver_like_dusting  \
0          0.0          0.0               0.0                  0.0   
1          0.0          0.0               0.0                  0.0   
2          0.0          0.0               0.0                  0.0   
3          0.0          0.0               0.0                  0.0   
4          0.0          0.0               0.0                  0.0   
...        ...          ...               ...                  ...   
4915       1.0          0.0               0.0                  0.0   
4916       0.0          0.0               0.0                  0.0   
4917       0.0          0.0               0.0                  0.0   
4918       0.0          0.0               0.0                  1.0   
4919       0.0          0.0               0.0                  0.0   

      extra_marital_contacts  itching  obesity  joint_pain  brittle_nails  \
0                        0.0      1.0      0.0         0.0            0.0   
1    

In [8]:
objList = data.select_dtypes(include = "object").columns
print (objList)

Index(['disease'], dtype='object')


In [9]:
print (data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4920 entries, 0 to 4919
Columns: 173 entries, vomiting to disease
dtypes: float64(172), object(1)
memory usage: 6.5+ MB
None


In [10]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for feat in objList:
    data[feat] = le.fit_transform(data[feat].astype(str))

print (data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4920 entries, 0 to 4919
Columns: 173 entries, vomiting to disease
dtypes: float64(172), int32(1)
memory usage: 6.5 MB
None


In [11]:
symp=[]
disease=[]
for i in range(len(data)):
    symp.append(data.columns[data.iloc[i]==1].to_list())
    disease.append(data.iloc[i,-1])

In [12]:
symp[0]

['itching',
 'skin_rash',
 'dischromic _patches',
 'nodal_skin_eruptions',
 'Fungal infection']

In [13]:
all_symp_col=list(data.columns[:-1])
def clean_symp(sym):
    return sym.replace('_',' ').replace('.1','').replace('(typhos)','').replace('yellowish','yellow').replace('yellowing','yellow') 

In [14]:
all_symp=[clean_symp(sym) for sym in (all_symp_col)]

In [15]:
found_syns=[]
notFound_syns=[]
for sym in all_symp:
    if not wn.synsets(sym) :
        notFound_syns.append(sym)
    else:
        found_syns.append(sym)

In [16]:
print(f'Found disease in wordnet {len(found_syns)}')

Found disease in wordnet 52


In [17]:
print(f'Not found disease in wordnet {len(notFound_syns)}')

Not found disease in wordnet 120


#### CREATING KNeighborsClassifier for prediction

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.3, random_state=0,stratify=y)

In [19]:
y_train.head()

,disease
959,Paralysis (brain hemorrhage)
511,Hypertension
3765,Osteoarthristis
923,Hypertension
2010,Acne


In [20]:
disease = data[["disease"]]

In [21]:
X_train.head()

,vomiting,indigestion,internal_itching,silver_like_dusting,extra_marital_contacts,itching,obesity,joint_pain,brittle_nails,excessive_hunger,...,Hyperthyroidism,Hypoglycemia,Osteoarthristis,Arthritis,(vertigo) Paroymsal Positional Vertigo,Acne,Urinary tract infection,Psoriasis,Impetigo,disease
959,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32
511,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23
3765,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,31
923,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23
2010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2


## Decision Tree

In [22]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
tree = DecisionTreeClassifier()
tree_params = [{'max_depth': list(range(5, 15)), 'criterion': ["entropy","gini"]}]
clf = GridSearchCV(tree, tree_params, cv = 10, scoring='accuracy')
clf.fit(X_train, y_train)
print(clf.best_params_)
print(clf.best_score_)

{'criterion': 'entropy', 'max_depth': 6}
1.0


In [23]:
decision_tree = DecisionTreeClassifier(max_depth=9,criterion="entropy")
decision_tree.fit(X_train,y_train)
print("Accuracy on training: {:.2f}".format(decision_tree.score(X_train, y_train)))
print("Accuracy on test: {:.2f}".format(decision_tree.score(X_test, y_test)))

Accuracy on training: 1.00
Accuracy on test: 1.00


In [24]:
print(classification_report(y_test,decision_tree.predict(X_test)))

                                         precision    recall  f1-score   support

(vertigo) Paroymsal  Positional Vertigo       1.00      1.00      1.00        36
                                   AIDS       1.00      1.00      1.00        36
                                   Acne       1.00      1.00      1.00        36
                    Alcoholic hepatitis       1.00      1.00      1.00        36
                                Allergy       1.00      1.00      1.00        36
                              Arthritis       1.00      1.00      1.00        36
                       Bronchial Asthma       1.00      1.00      1.00        36
                   Cervical spondylosis       1.00      1.00      1.00        36
                            Chicken pox       1.00      1.00      1.00        36
                    Chronic cholestasis       1.00      1.00      1.00        36
                            Common Cold       1.00      1.00      1.00        36
                           

## KNN

In [25]:
from sklearn.neighbors import KNeighborsClassifier  
knn = KNeighborsClassifier()
knn_params= {"n_neighbors":list(range(1, 20))}
clf4 = GridSearchCV(knn, knn_params, cv = 10, scoring='accuracy')
clf4.fit(X_train, y_train)
print(clf4.best_params_)
print(clf4.best_score_)

{'n_neighbors': 1}
1.0


In [26]:
knn_model_1 = KNeighborsClassifier(n_neighbors=1) 
knn_model_1.fit(X_train, y_train) 
print("Accuracy on training: {:.2f}".format(knn_model_1.score(X_train, y_train)))
print("Accuracy on test: {:.2f}".format(knn_model_1.score(X_test, y_test)))

Accuracy on training: 1.00
Accuracy on test: 1.00


In [32]:
# save the model to disk
filename = 'final_model.sav'
joblib.dump(knn_model_1, filename)

['final_model.sav']

In [ ]:
# load the model from disk
loaded_model = joblib.load(filename)
result = loaded_model.score(X_test, Y_test)
print(result)

In [27]:
print(classification_report(y_test,knn_model_1.predict(X_test)))

                                         precision    recall  f1-score   support

(vertigo) Paroymsal  Positional Vertigo       1.00      1.00      1.00        36
                                   AIDS       1.00      1.00      1.00        36
                                   Acne       1.00      1.00      1.00        36
                    Alcoholic hepatitis       1.00      1.00      1.00        36
                                Allergy       1.00      1.00      1.00        36
                              Arthritis       1.00      1.00      1.00        36
                       Bronchial Asthma       1.00      1.00      1.00        36
                   Cervical spondylosis       1.00      1.00      1.00        36
                            Chicken pox       1.00      1.00      1.00        36
                    Chronic cholestasis       1.00      1.00      1.00        36
                            Common Cold       1.00      1.00      1.00        36
                           

In [28]:
y_pred = knn_model_1.predict(X_test)

In [29]:
y_pred

array(['Hypertension ', 'Hypoglycemia', 'hepatitis A', ...,
       'Osteoarthristis', 'Acne', 'Osteoarthristis'], dtype=object)